In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [2]:
dftrain = pd.read_csv('https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/jobmadrid/dataset/jm_train.csv')
dftest = pd.read_csv('https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/jobmadrid/dataset/jm_X_test.csv')

Para empezar hice un RandomForest por default con los que me salió un f1_score de 77% aprox con profundidad de entre 17-19 y unas 300 hojas. Probablemente overfitteado, por lo que paso a hacer un grid search con parámetros basados en lo que se observa.

Además de revisar los datos, el balanceo de clases, correlaciones entre parámetros con cada clase y la matriz de confusión de un RandomForest básico, parece que hay más fallos con las clases 1 y 2.

In [3]:
y = dftrain.target
X = dftrain.drop('target', axis=1)
X2 = np.array(X) #para kfold

In [4]:
#Realizamos búsqueda de mejores parámetros para evitar overfitting

clf = RandomForestClassifier()
parametros = {'n_estimators':[201],
              'criterion':['entropy'],
             'max_depth':[7,9,11,13,14],
             'n_jobs':[-1],
              'max_leaf_nodes': range(20,100,5),
              'min_impurity_decrease' : [0.01]
             }

grid = GridSearchCV(estimator = clf,
                    param_grid = parametros,
                    scoring = 'f1_macro',
                    n_jobs = -1 )

search_result = grid.fit(X,y)
search_result.best_params_

{'criterion': 'entropy',
 'max_depth': 13,
 'max_leaf_nodes': 90,
 'min_impurity_decrease': 0.01,
 'n_estimators': 201,
 'n_jobs': -1}

In [6]:
#MODELO AVERAGE. Uso de Kfold

kfoldlist = []
y = dftrain['target']
X2 = np.array(dftrain.drop('target', axis=1))
kf = StratifiedKFold()
for train_index, test_index in kf.split(X2, y):
    X_train, X_test = X2[train_index], X2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    average = search_result.best_estimator_
    

    m_av = search_result.best_estimator_
    m_av = m_av.fit(X_train, y_train)
    yhat = m_av.predict(X_test)
    kfoldlist.append(f1_score(y_test,yhat, average='macro'))

    
m_av.fit(X2,y)
print(sum(kfoldlist)/len(kfoldlist))



0.7338869643903839


Puesto que el F1_score de la gridsearch es peor que el default, y viendo que efectivamente falla más con las clases 1 y 2. Probablemente no esté overfit.

In [7]:
#MODELO PONDERADO. Uso de Kfold.
c = dftrain.copy()
b = dftrain[dftrain['target']==2].sample(100) #Mirando los datos de clases y                                                 
f = dftrain[dftrain['target']==1].sample(70)  #fallos decido añadir 100 de una y 70 de otra
d = pd.concat([b,c,f], axis=0, ignore_index=True)

X2 = np.array(d.drop(['target'], axis=1))
y = d['target']

kfoldlist = []

for train_index, test_index in kf.split(X2, y):
    X_train, X_test = X2[train_index], X2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ponderado = search_result.best_estimator_
    

    m_pond = search_result.best_estimator_
    m_pond = m_pond.fit(X_train, y_train)
    yhat = m_pond.predict(X_test)
    kfoldlist.append(f1_score(y_test,yhat, average='macro'))
    
m_pond.fit(X2,y)    
print(sum(kfoldlist)/len(kfoldlist))


0.7392940139531262


Hago un pequeño oversampling de las clases minoritarias con idea de ponderar un tercer modelo (basándome en los dos anteriores)

In [10]:
###Conjunción de los modelos

yhatpremium=[] #Será la yhat de los modelos anteriores
avhat = m_av.predict_proba(dftest)  #modelo average
ponhat = m_pond.predict_proba(dftest)#modelo ponderado


for av,pon in zip(avhat, ponhat):
    clase=list(av)
    maximo = max(av)
    
    if maximo > 0.6:  #Si el modelo average está muy seguro (0.6%) lo aceptamos
        yhatpremium.append(clase.index(maximo))
        
    elif clase.index(maximo) in [1,2]: #en caso contrario, si iba a decidir que era de la clase 1 ó 2,
            matrix=[]
            for i in range(5):         #hacemos una ponderación de cada clase entre los dos modelos
                matrix.append(av[i]*0.15+pon[i]*0.85)#donde el modelo ponderado tiene mucho más peso que el average
            pond_max = max(matrix)
                
            if pond_max>0.5:         #si tiene una seguridad aceptable (0.5%) lo aceptamos como predicción
                yhatpremium.append(matrix.index(pond_max))
                
            else:                     #pero si ni aún con la ponderación tiene cierta seguridad, nos decantamos por
                yhatpremium.append(clase.index(maximo)) #el modelo original
    else:
        yhatpremium.append(clase.index(maximo)) #si el modelo original no iba a predecir una clase 1 ó 2 lo mantenemos.
        
yhat = np.array(yhatpremium)
yhat

/Users/adri/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/adri/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([1, 3, 2, 0, 3, 2, 4, 0, 2, 3, 2, 4, 3, 1, 4, 4, 4, 4, 2, 0, 1, 2,
       3, 0, 0, 3, 0, 2, 2, 4, 2, 2, 4, 3, 3, 1, 0, 4, 3, 0, 4, 2, 3, 2,
       2, 0, 1, 1, 2, 4, 4, 0, 2, 0, 4, 4, 1, 3, 3, 0, 3, 0, 1, 0, 4, 1,
       1, 0, 2, 3, 0, 1, 4, 0, 4, 0, 0, 3, 2, 3, 4, 3, 1, 4, 1, 1, 2, 4,
       1, 2, 2, 4, 1, 1, 0, 1, 4, 2, 1, 4, 0, 0, 0, 4, 1, 4, 4, 4, 1, 2,
       4, 1, 3, 3, 3, 2, 3, 4, 2, 0, 0, 4, 4, 3, 3, 2, 4, 1, 3, 2, 2, 1,
       4, 0, 0, 0, 3, 0, 2, 4, 0, 4, 3, 2, 1, 3, 4, 3, 4, 4, 3, 2, 0, 2,
       0, 4, 1, 4, 4, 2, 0, 1, 3, 1, 1, 4, 1, 3, 4, 1, 1, 2, 4, 4, 1, 2,
       3, 1, 3, 4, 2, 3, 0, 4, 2, 2, 0, 3, 4, 1, 1, 3, 2, 4, 0, 0, 4, 4,
       1, 4, 0, 2, 4, 3, 2, 2, 3, 1, 1, 3, 0, 0, 2, 1, 1, 3, 4, 0, 2, 1,
       2, 3, 2, 0, 3, 2, 0, 2, 2, 4, 0, 2, 4, 3, 4, 4, 0, 3, 3, 3, 1, 4,
       3, 0, 2, 0, 2, 1, 2, 0, 4, 3, 1, 3, 1, 1, 0, 3, 2, 4, 2, 1, 0, 2,
       2, 4, 2, 4, 1, 2, 1, 4, 4, 0, 3, 0, 1, 4, 2, 4, 1, 1, 2, 1, 1, 4,
       1, 0, 2, 2, 2, 1, 0, 2, 0, 0, 4, 4, 1, 4, 0,

Tras las validaciones de f1_score, el modelo ponderado obtiene un 0.79 con muchas menos hojas que el RandomForest por default y evitando overfittings.